In [1]:
import torch
from transformers import AutoTokenizer, AutoProcessor
import vllm
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import os
import pandas as pd

# os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

model_path = "/data/model/Qwen/Qwen2-VL-2B-Instruct-AWQ"
data_path = "/data/MMRetrieval/data/marqo.json"
image_dir = "/data/MMRetrieval/data/train_images"

print(torch.cuda.device_count())

0


/opt/conda/lib/python3.11/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
def generate_pair_prompt(image_path_1, text_1, image_path_2, text_2):
    message = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are now a product expert. You will be shown two images of products and their descriptions. You need to judge whether the two images are the same product or not.",
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"Product 1: {text_1}"},
                {
                    "type": "image",
                    "image": image_path_1,
                },
                {"type": "text", "text": f"Product 2: {text_2}"},
                {"type": "image", "image": image_path_2},
            ],
        },
    ]

    return message


def generate_prompt(row, df):
    query_text = row["title"]
    query_image_path = row["image_path"]

    prompts = []

    for pred_id in row["pred"]:
        pred_row = df.loc[pred_id]
        pred_text = pred_row["title"]
        pred_image_path = pred_row["image_path"]

        prompt = generate_pair_prompt(
            query_image_path, query_text, pred_image_path, pred_text
        )
        prompts.append(prompt)

    return prompts

In [4]:
processor = AutoProcessor.from_pretrained(model_path)

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    dtype="half",
    max_model_len=10000,
    disable_log_stats=True,
    device="cuda",
)

INFO 01-09 10:11:25 config.py:510] This model supports multiple tasks: {'classify', 'embed', 'generate', 'score', 'reward'}. Defaulting to 'generate'.
WARNING 01-09 10:11:26 config.py:588] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 01-09 10:11:26 config.py:1310] Defaulting to use mp for distributed inference


NotImplementedError: 

In [6]:
df = pd.read_json(data_path, lines=True)
df["image_path"] = df["image_phash"].apply(lambda x: os.path.join(image_dir, x))
df.head(1)

,posting_id,image,image_phash,title,label_group,pred,image_path
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361, train_1147...",/data/MMRetrieval/data/train_images/94974f937d...
